# Add slice of zeros to a bunch of the forcing files

In [91]:
import numpy as np
import xarray as xr
import glob
import matplotlib.pyplot as plt
import glob

#### Functions

In [ ]:
# https://stackoverflow.com/questions/33435953/is-it-possible-to-append-to-an-xarray-dataset

### Main

In [77]:
domain_old   = xr.open_dataset(f'/gws/nopw/j04/terrafirma/birgal/NEMO_AIS/bathymetry/domain_cfg-old.nc')
domain_new   = xr.open_dataset(f'/gws/nopw/j04/terrafirma/birgal/NEMO_AIS/bathymetry/domain_cfg-20231025.nc')
bergmelt_old = xr.open_dataset(f'/gws/nopw/j04/terrafirma/birgal/NEMO_AIS/output/old-forcing/bergmelt.nc')
calving_old  = xr.open_dataset(f'/gws/nopw/j04/terrafirma/birgal/NEMO_AIS/output/old-forcing/calving.nc')
bfrcoef_old  = xr.open_dataset(f'/gws/nopw/j04/terrafirma/birgal/NEMO_AIS/output/old-forcing/bfr_coef.nc')
runoff_old   = xr.open_dataset(f'/gws/nopw/j04/terrafirma/birgal/NEMO_AIS/output/old-forcing/runoff_y1979.nc')
shlat2d_old  = xr.open_dataset(f'/gws/nopw/j04/terrafirma/birgal/NEMO_AIS/output/old-forcing/shlat2d.nc')

In [ ]:
# add slice of points nort of the pre-existing points

In [89]:
print(bergmelt_old.berg_melt.dims)
print(calving_old.soicbclv.dims)
print(bfrcoef_old.bfr_coef.dims)
print(shlat2d_old.shlat2d.dims)

('time_counter', 'y', 'x')
('time_counter', 'y', 'x')
('time_counter', 'y', 'x')
('y', 'x')


In [141]:
vars  = ['berg_melt', 'bfr_coef', 'soicbclv', 'shlat2d']
folder_out = '/gws/nopw/j04/terrafirma/birgal/NEMO_AIS/output/old-forcing/extended/'
files = glob.glob('/gws/nopw/j04/terrafirma/birgal/NEMO_AIS/output/old-forcing/*.nc')
for var in vars:
    print(var)
    for file in files:
        ds = xr.open_dataset(file)
        if var in ds:
            try:
                new_var = np.empty((ds.time_counter.shape[0], domain_new.nav_lon.shape[0], domain_new.nav_lon.shape[1]))
                new_var[:] = np.nan
                new_var[:,:439,:] = ds[var].values
                
                ds_new = xr.Dataset(data_vars = {var:(["time_counter", "y", "x"], new_var)},
                                    coords    = {'nav_lon': domain_new.nav_lon,
                                                 'nav_lat': domain_new.nav_lat,
                                                 'time_counter': ds.time_counter})
                ds_new.to_netcdf(f"{folder_out}{file.split('old-forcing/')[1].split('.')[0]}-extended.nc")
            except:
                new_var = np.empty((domain_new.nav_lon.shape[0], domain_new.nav_lon.shape[1]))
                new_var[:] = np.nan
                new_var[:439,:] = ds[var].values
                
                ds_new = xr.Dataset(data_vars = {var:(["y", "x"], new_var)},
                                    coords    = {'nav_lon': domain_new.nav_lon,
                                                 'nav_lat': domain_new.nav_lat})
                new_name = file.split('old-forcing/')
                ds_new.to_netcdf(f"{folder_out}{file.split('old-forcing/')[1].split('.')[0]}-extended.nc")

berg_melt
bfr_coef
soicbclv
shlat2d
